# Setting Up unsloth and dependencies

In [6]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy; get_numpy = f"numpy=={numpy.__version__}"
    except: get_numpy = "numpy"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} torchvision bitsandbytes "transformers==4.56.2" trackio \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth trackio
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

# Install the environment and custom openEnv 

In [9]:
%%capture
!pip install -qqq fastapi uvicorn requests
!git clone https://github.com/yogesh1801/OpenEnv.git > /dev/null 2>&1
%cd OpenEnv
!git checkout yogesh-julia-env

In [3]:
import subprocess, sys, os
from pathlib import Path
sys.path.insert(0, './src')
working_directory = str(Path.cwd().parent.absolute() / "OpenEnv")

# Setting up a Model using unsloth which we will train

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1500
lora_rank = 10
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    fast_inference = True,
    load_in_4bit = False,
    max_seq_length = max_seq_length,
    gpu_memory_utilization = 0.85,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
INFO 10-28 11:56:14 [__init__.py:225] Automatically detected platform rocm.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: AMD currently is not stable with 4bit bitsandbytes. Disabling for now.


[2025-10-28 11:56:17] INFO vllm_utils.py:752: Unsloth: Patching vLLM


INFO 10-28 11:56:17 [vllm_utils.py:694] Unsloth: Patching vLLM v1 graph capture
Unsloth: Could not patch vLLM V0 graph capture: No module named 'vllm.worker'
==((====))==  Unsloth 2025.10.9: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.11.1rc3.dev39+gf417746ad.rocm700.
   \\   /|    . Num GPUs = 1. Max memory: 191.688 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0a0+git1c57644. ROCm Toolkit: 7.0.51831-a3e329ad8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 84.85%
Unsloth: Your GPU has CUDA compute capability 9.4 with VRAM = 191.69 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1500. Num Sequences = 400.
Unsloth: vLLM's KV Cache can use up to 156.48 GB. Also swap space = 6 GB.
Unsloth: Not an error,

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 10-28 11:56:24 [default_loader.py:314] Loading weights took 2.63 seconds
INFO 10-28 11:56:24 [punica_selector.py:20] Using PunicaWrapperGPU.
INFO 10-28 11:56:24 [gpu_model_runner.py:2917] Model loading took 7.0195 GiB and 3.096526 seconds
INFO 10-28 11:56:29 [backends.py:609] Using cache directory: /root/.cache/vllm/torch_compile_cache/9f1bce1ea9/rank_0_0/backbone for vLLM's torch.compile
INFO 10-28 11:56:29 [backends.py:623] Dynamo bytecode transform time: 4.67 s
INFO 10-28 11:56:31 [backends.py:207] Directly load the compiled graph(s) for dynamic shape from the cache, took 1.306 s
INFO 10-28 11:56:45 [monitor.py:34] torch.compile takes 5.98 s in total
INFO 10-28 11:56:45 [gpu_worker.py:337] Available KV cache memory: 153.07 GiB
INFO 10-28 11:56:46 [kv_cache_utils.py:1229] GPU KV cache size: 1,433,040 tokens
INFO 10-28 11:56:46 [kv_cache_utils.py:1234] Maximum concurrency for 1,500 tokens per request: 952.82x
INFO 10-28 11:56:46 [vllm_utils.py:699] Unsloth: Running patched vLLM v

...

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:07<00:00, 17.57it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:06<00:00, 16.93it/s]

INFO 10-28 11:56:59 [gpu_model_runner.py:3843] Graph capturing finished in 14 secs, took 0.83 GiB
INFO 10-28 11:56:59 [vllm_utils.py:706] Unsloth: Patched vLLM v1 graph capture finished in 14 secs.


INFO 10-28 11:57:00 [core.py:238] init engine (profile, create kv cache, warmup model) took 36.00 seconds
INFO 10-28 11:57:01 [llm.py:343] Supported tasks: ('generate',)
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'layer_norm2', 'post_layernorm', 'input_layernorm', 'q_norm', 'post_feedforward_layernorm', 'k_norm', 'layer_norm1', 'attention_norm', 'post_attention_layernorm', 'norm1', 'norm2', 'ffn_norm']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'layer_norm2', 'post_layernorm', 'cross_attn_post_attention_layernorm', 'input_layernorm', 'q_norm', 'post_feedforward_layernorm', 'k_norm', 'cross_attn_input_layernorm', 'layer_norm1', 'attention_norm', 'post_attention_layernorm', 'norm1', 'norm2', 'ffn_norm']


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

Unsloth 2025.10.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Connecting to Zig Env exposed by OpenEnv

In [6]:
from envs.zig_env import ZigEnv
from envs.zig_env.models import ZigAction, ZigObservation

1. It is not recommended to directly open a subprocess in python instead expose OpenEnv using a docker container because u know that is the whole point of OpenEnv

### First, build the OpenEnv base image (one-time setup):

```bash
# From OpenEnv root directory
docker build -t openenv-base:latest -f src/core/containers/images/Dockerfile .
```

### Build Zig Environment Image

```bash
# From OpenEnv root directory
docker build -t zig-env:latest -f src/envs/zig_env/server/Dockerfile .
```

### Run the Server

```bash
# Run in background
docker run -d -p 8000:8000 --name zig-env-server zig-env:latest

# OR run in foreground (to see logs)
docker run -p 8000:8000 --name zig-env-server zig-env:latest
```


Note this has to be done in system terminal, since this notebook is hosted inside the docker running these commands in terminal exposed by this notebook is illogical as you cannot run docker inside a docker

In [1]:
port = "8003"
localhost = f"http://localhost:{port}"
# openenv_process = subprocess.Popen(
#     [sys.executable, "-m", "uvicorn", "envs.julia_env.server.app:app", "--host", "0.0.0.0", "--port", port],
#     env = {**os.environ,
#          "PYTHONPATH": f"{working_directory}/src",
#          },
#     stdout = subprocess.PIPE,
#     stderr = subprocess.PIPE,
#     text = True,
#     cwd = working_directory,
# )


# used docker images instead

In [8]:
import requests
import time
print(requests.get(f"{localhost}/health", timeout = 2).content)
openenv_process = ZigEnv(base_url = localhost)

b'{"status":"healthy"}'


In [9]:
result = openenv_process.reset()
current_state = result.observation
current_state

action = ZigAction(
    core_code="""const std = @import("std");

pub fn add(a: i32, b: i32) i32 {
    return a + b;
}""",
    test_code="""const std = @import("std");
const testing = std.testing;

test "add with positive numbers" {
    try testing.expectEqual(@as(i32, 3), add(1, 2));
}

test "add with negative numbers" {
    try testing.expectEqual(@as(i32, -1), add(-3, 2));
}

test "add with zeros" {
    try testing.expectEqual(@as(i32, 0), add(0, 0));
}"""
)

result = openenv_process.step(action)
result

StepResult(observation=JuliaObservation(done=False, reward=None, metadata={}, stdout='Add function Tests: Error During Test at /tmp/tmpamvekpcc.jl:19\n  Test threw exception\n  Expression: Project.add(1, 2) == 3\n  UndefVarError: `Project` not defined\n  Stacktrace:\n   [1] macro expansion\n     @ ~/.julia/juliaup/julia-1.10.10+0.x64.linux.gnu/share/julia/stdlib/v1.10/Test/src/Test.jl:669 [inlined]\n   [2] macro expansion\n     @ /tmp/tmpamvekpcc.jl:19 [inlined]\n   [3] macro expansion\n     @ ~/.julia/juliaup/julia-1.10.10+0.x64.linux.gnu/share/julia/stdlib/v1.10/Test/src/Test.jl:1577 [inlined]\n   [4] top-level scope\n     @ /tmp/tmpamvekpcc.jl:17\nTest Summary:      | Pass  Error  Total  Time\nAdd function Tests |    2      1      3  0.7s\n', stderr='ERROR: LoadError: Some tests did not pass: 2 passed, 0 failed, 1 errored, 0 broken.\nin expression starting at /tmp/tmpamvekpcc.jl:16\n', exit_code=1, tests_passed=2, tests_failed=1, code_compiles=True), reward=7, done=False)

# Training starts here

In [10]:
zig_code_gen_prompt = """
You are a precise and pragmatic Zig programmer.
Write a **single Zig function** that correctly solves the problem described below.
Rules:
- The code must be syntactically correct and runnable as is.
- Use proper Zig syntax with explicit types and return statements.
- Use only the Zig standard library.
- Do **not** wrap the code in a module or add a `main` function.
- Do **not** include any test code in your response.
- Do **not** hardcode specific test cases or outputs — the function must work for general inputs.
- The **function name must exactly match** the one used in the provided tests.
- Respond with **only the Zig function** and nothing else (no explanations, no comments, no extra text)
- The function name must exactly match the one used in the provided tests.
- Return only the Zig function.
- Use proper Zig types (i32, i64, f32, f64, []const u8, etc.)
- Include proper error handling with try/catch if needed.
- Use semicolons at the end of statements.
- Remember Zig requires explicit return statements.
Passing tests and clean, compilable code are rewarded. Hardcoding or failing tests is penalized.
Test Reference (for context only, do not include in the output):
{zig_test}
Code:
""".strip()

In [11]:
import re
def remove_ticks(text):
    text = re.sub(r'^```zig\s*\n?', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\n?```\s*$', '', text)

    return text

In [12]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    temperature = 1.0,
    learning_rate = 5e-5,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 2,
    max_prompt_length = 2048,
    max_completion_length = 1024,
    num_train_epochs = 1,
    max_steps = 500,
    save_steps = 250,
    report_to = "trackio",
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


In [13]:
import pandas as pd
from datasets import Dataset

df = pd.read_parquet("../zig_trainset.parquet")

df = df[["zig_prompt", "zig_test", "first_test_case"]]
dataset = Dataset.from_pandas(df)

dataset = dataset.map(lambda x: {
    "prompt": [
        {"role" : "system", "content": zig_code_gen_prompt.format(zig_test=x["first_test_case"])},
        {"role" : "user", "content": x["zig_prompt"]},
    ],
})
print(dataset[0])

Map:   0%|          | 0/6326 [00:00<?, ? examples/s]

{'julia_prompt': 'Given a vector of integers, write a function `find_max_sum_subarray(arr::Vector{Int64})` that returns the maximum sum of any contiguous subarray within the vector. A subarray is a continuous portion of the vector. You may assume that the input vector will always have at least one element.', 'julia_test': 'using Test\n\n@testset "my test" begin\n@test find_max_sum_subarray([1, 2, 3, 4, 5]) == 15\n@test find_max_sum_subarray([-1, -2, -3, -4, -5]) == -1\n@test find_max_sum_subarray([1, -2, 3, -4, 5]) == 5\n@test find_max_sum_subarray([-2, 1, -3, 4, -1, 2, 1, -5, 4]) == 6\nend', 'first_test_case': 'using Test\n\n@testset "my test 1" begin\n@test find_max_sum_subarray([1, 2, 3, 4, 5]) == 15\nend', 'prompt': [{'content': 'You are a precise and pragmatic Julia programmer.\n\nWrite a **single Julia function** that correctly solves the problem described below.\n\nRules:\n- The code must be syntactically correct and runnable as is.\n- Do not use arrow functions, ternary operato

In [14]:
def zig_env_reward(completions, **kwargs):
    rewards = []
    test_codes_list = kwargs.get('zig_test', [])
    
    for i, completion in enumerate(completions):
        response = completion[0]["content"]
        core_code = remove_ticks(response)
        test_code = test_codes_list[i]
        
        try:
            result = openenv_process.reset()
            action = ZigAction(core_code=core_code, test_code=test_code)
            result = openenv_process.step(action)

            reward = result.reward if result.reward is not None else 0.0
            
        except Exception as e:
            print(f"Error for completion {i}:", e)
            reward = 0.0
        
        rewards.append(reward)
    
    return rewards

In [15]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    remove_unused_columns = False,
    reward_funcs = [zig_env_reward],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)

In [16]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128004}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,326 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 15,196,160 of 3,227,945,984 (0.47% trained)


* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/.cache/huggingface/trackio


* Created new run: patient-brook-35
WARNING 10-28 11:58:15 [processor.py:228] vLLM has deprecated support for supporting different tokenizers for different LoRAs. By default, vLLM uses base model's tokenizer. If you are using a LoRA with its own tokenizer, consider specifying `--tokenizer [lora_path]` to use the LoRA tokenizer.


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / julia_env_reward / mean,rewards / julia_env_reward / std
1,0.000000,0.000000,2.828427,49.500000,43.000000,56.000000,0.000000,49.500000,43.000000,56.000000,0,0,0,0,0,0.000322,0.000000,2.828427
2,0.000000,7.000000,12.727922,126.500000,125.000000,128.000000,0.000000,126.500000,125.000000,128.000000,No Log,No Log,No Log,No Log,No Log,0.000190,7.000000,12.727922
3,0.000000,10.000000,0.000000,58.000000,38.000000,78.000000,0.000000,58.000000,38.000000,78.000000,No Log,No Log,No Log,No Log,No Log,0.000041,10.000000,0.000000
4,0.000000,16.000000,0.000000,62.000000,61.000000,63.000000,0.000000,62.000000,61.000000,63.000000,No Log,No Log,No Log,No Log,No Log,0.000021,16.000000,0.000000
5,0.000000,16.000000,0.000000,54.000000,52.000000,56.000000,0.000000,54.000000,52.000000,56.000000,No Log,No Log,No Log,No Log,No Log,0.000080,16.000000,0.000000
6,0.000000,16.000000,0.000000,66.500000,62.000000,71.000000,0.000000,66.500000,62.000000,71.000000,No Log,No Log,No Log,No Log,No Log,0.000116,16.000000,0.000000
7,0.000000,16.000000,0.000000,49.500000,47.000000,52.000000,0.000000,49.500000,47.000000,52.000000,No Log,No Log,No Log,No Log,No Log,0.000251,16.000000,0.000000
8,0.000000,4.000000,8.485281,187.000000,147.000000,227.000000,0.000000,187.000000,147.000000,227.000000,No Log,No Log,No Log,No Log,No Log,0.000120,4.000000,8.485281
9,0.000000,2.000000,5.656854,138.000000,87.000000,189.000000,0.000000,138.000000,87.000000,189.000000,No Log,No Log,No Log,No Log,No Log,0.000851,2.000000,5.656854
10,0.000000,-2.000000,0.000000,92.500000,86.000000,99.000000,0.000000,92.500000,86.000000,99.000000,No Log,No Log,No Log,No Log,No Log,0.000355,-2.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!
* Run finished. Uploading logs to Trackio (please wait...)


TrainOutput(global_step=500, training_loss=0.0003393380002628046, metrics={'train_runtime': 2032.0529, 'train_samples_per_second': 0.492, 'train_steps_per_second': 0.246, 'total_flos': 0.0, 'train_loss': 0.0003393380002628046})

# Post training and inferencing

In [19]:
model.save_lora("grpo_saved_lora_zig")

In [20]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora_zig/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

In [23]:
import pandas as pd
from datasets import Dataset

df = pd.read_parquet("../zig_testset.parquet")

df = df[["zig_prompt", "zig_test", "task_id", "first_test_case"]]
dataset = Dataset.from_pandas(df)

MAX_LEN = 1500

dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": zig_code_gen_prompt.format(zig_test=x["first_test_case"])[:MAX_LEN]},
        {"role": "user", "content": x["zig_prompt"][:MAX_LEN]},
    ],
})

dataset[0]

Map:   0%|          | 0/2712 [00:00<?, ? examples/s]

{'julia_prompt': 'You are tasked with creating a function that simulates the creation of a tree node. The function should take a context string and a variable number of key-value pairs representing node attributes. It should return a dictionary representing the created node, with the context as a key-value pair and the additional attributes included as part of the dictionary. Implement the function `create_node(context::String, attributes...)` where `context` is a string and `attributes` are the key-value pairs of the node attributes. The output should be a dictionary containing the context and the attributes provided. For example, if the input is `create_node("root", "color" => "red", "size" => "10")`, the output should be `Dict("context" => "root", "color" => "red", "size" => "10")`.',
 'julia_test': 'using Test\n\n@testset "my test" begin\n@test create_node("root") == Dict("context" => "root")\n@test create_node("root", "color" => "red") == Dict("context" => "root", "color" => "red"

In [24]:
import pandas as pd
from tqdm import tqdm
from vllm import SamplingParams

sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)

results = []
for i in tqdm(range(len(dataset)), desc="Generating Zig Code"):
    task_id = dataset[i]["task_id"]
    zig_test = dataset[i]["zig_test"]
    message = dataset[i]["prompt"]
    text = tokenizer.apply_chat_template(
        message,
        add_generation_prompt=True,
        tokenize=False,
    )
    response = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_saved_lora_zig"),
    )
    zig_code = response[0].outputs[0].text
    zig_code = remove_ticks(zig_code)
    results.append({
        "task_id": task_id,
        "zig_code": zig_code,
        "zig_test": zig_test
    })

# Save results to parquet
result_df = pd.DataFrame(results)
result_df.to_parquet("zig_code_output_smaller_model.parquet", index=False)
print("✅ Zig code generation completed and saved to zig_code_output_smaller_model.parquet")

Generating Julia Code:   0%|                                                                                                                                                                                                                          | 0/2712 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Generating Julia Code:   0%|                                                                                                                                                                                                                  | 1/2712 [00:00<17:27,  2.59it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

...

Generating Julia Code: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2711/2712 [25:28<00:00,  1.19it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|                                                                                      …

Generating Julia Code: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2712/2712 [25:30<00:00,  1.77it/s]

✅ Julia code generation completed and saved to julia_code_output_smaller_model.parquet
